In [1]:
from collections import deque, defaultdict

import numpy as np

In [2]:
directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

In [3]:
def parse_input(file):

    with open(file) as file_in:
        grid_str = file_in.read()
    grid = np.array([list(row) for row in grid_str.splitlines()])
    grid = np.pad(grid, pad_width=1, mode='constant', constant_values='.')

    return grid

In [4]:
def get_region_plant_start(grid, plant, start):

    assert grid[start] == plant

    n_rows, n_cols = grid.shape

    visited = set()
    n_connexions = 0
    queue = deque([start])

    while queue:
        x_current, y_current = queue.popleft()
        if (x_current, y_current) in visited:
            continue
        visited.add((x_current, y_current))

        for dx, dy in directions:
            x_next, y_next = x_current + dx, y_current + dy
            if 0 <= x_next < n_rows and 0 <= y_next < n_cols and (x_next, y_next) not in visited:
                if grid[x_next, y_next] == plant:
                    queue.append((x_next, y_next))
                    n_connexions += 1

    return visited, n_connexions

In [5]:
def get_all_regions_one_plant(grid, plant):
    plant_pos = set([tuple(pos.tolist()) for pos in np.argwhere(grid == plant)])
    plant_regions = []

    while plant_pos:
        start = plant_pos.pop()
        region, n_connexions = get_region_plant_start(grid, plant, start)
        plant_regions.append((str(plant), n_connexions, region))
        plant_pos = plant_pos.difference(region)

    return plant_regions

In [6]:
def get_all_regions(grid):
    regions = []
    for plant in np.unique(grid):
        if plant != '.':
            regions.extend(get_all_regions_one_plant(grid, plant))
    return regions

In [24]:
def cound_sides_region(coords):

    n_corners = 0

    for x, y in coords:

        # Count exterior corners
        n_corners += (x-1, y) not in coords and (x, y-1) not in coords  # NW
        n_corners += (x-1, y) not in coords and (x, y+1) not in coords  # NE
        n_corners += (x+1, y) not in coords and (x, y+1) not in coords  # SE
        n_corners += (x+1, y) not in coords and (x, y-1) not in coords  # SW           

        # Count interior corners
        n_corners += (x+1, y) in coords and (x, y+1) in coords and (x+1, y+1) not in coords  # SE
        n_corners += (x-1, y) in coords and (x, y+1) in coords and (x-1, y+1) not in coords  # NE
        n_corners += (x+1, y) in coords and (x, y-1) in coords and (x+1, y-1) not in coords  # SW
        n_corners += (x-1, y) in coords and (x, y-1) in coords and (x-1, y-1) not in coords  # NW
            
    return n_corners

In [25]:
def compute_total_cost(grid, part):
    total_cost = 0
    for plant, n_connexions, coords in get_all_regions(grid):
        area = len(coords)
        if part == 1:
            factor = 4 * area - 2 * n_connexions  # perimeter
        elif part == 2:
            factor = cound_sides_region(coords)
        total_cost += area * factor
    return total_cost

In [26]:
def main(file, part):
    grid = parse_input(file)
    total_cost = compute_total_cost(grid, part)
    return total_cost

In [27]:
assert main('example1.txt', part=1) == 140
assert main('example2.txt', part=1) == 772
assert main('example3.txt', part=1) == 1930

In [28]:
main('input.txt', part=1)

1431440

In [33]:
assert main('example1.txt', part=2) == 80
assert main('example2.txt', part=2) == 436
assert main('example4.txt', part=2) == 236

In [34]:
main('input.txt', part=2)

869070